In [3]:
#https://www.kaggle.com/jonathanbouchet/clustering-starbucks-items

# ^------------------ data visualization reference



In [4]:
#remove heatmap redundancy

#https://medium.com/@dtse2401/formatting-tips-for-correlation-heatmaps-in-seaborn-4478ef15d87f



In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [6]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale


pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

pd.set_option('display.max_colwidth', None)
pd.options.display.max_info_columns = 999

#return the first 5 and last 5 rows of this dataframe
def p(df_):
    if df_.shape[0] > 6:
        print(df_.shape)
        return pd.concat([df_.head(), df_.tail()])
    else:
        return df_

def rcp(file_, pd_=None):
    if pd_ == None:
        return pd.read_csv(os.path.join('..', 'processed_data', file_))
    else:
        return pd.read_csv(os.path.join('..', 'processed_data', file_), parse_dates=pd_)
    
def rcr(file_, pd_=None):
    if pd_ == None:
        return pd.read_csv(os.path.join('..', 'raw_data', file_))
    else:
        return pd.read_csv(os.path.join('..', 'raw_data', file_), parse_dates=pd_)
    
#sort dataframe by column
def s(df_, column_):
    return df_.sort_values(column_)

#reset index and sort dataframe by column
def sr(df_, column_):
    df_ = df_.sort_values(column_)
    return df_.reset_index(drop=True)

#print length of list
def pl(list_):
    print(len(list_))
    return list_

#print length of list
def pdc(dict_):
    print(len(dict_))
    return dict_



In [7]:
files = os.listdir(os.path.join('..', 'raw_data'))
pl(files)

5


['starbucks-menu-nutrition-food.csv',
 'starbucks_drinkMenu_expanded.csv',
 '.DS_Store',
 'starbucks-menu-nutrition-food2.csv',
 'starbucks-menu-nutrition-drinks.csv']

In [8]:
files = [k for k in files if not k in ['.DS_Store', 'starbucks-menu-nutrition-food.csv']]
pl(files)

3


['starbucks_drinkMenu_expanded.csv',
 'starbucks-menu-nutrition-food2.csv',
 'starbucks-menu-nutrition-drinks.csv']

In [9]:
df_starbucks_drinkMenu_expanded = rcr('starbucks_drinkMenu_expanded.csv')
p(df_starbucks_drinkMenu_expanded)

(242, 18)


,Beverage_category,Beverage,Beverage_prep,Calories,Total Fat (g),Trans Fat (g),Saturated Fat (g),Sodium (mg),Total Carbohydrates (g),Cholesterol (mg),Dietary Fibre (g),Sugars (g),Protein (g),Vitamin A (% DV),Vitamin C (% DV),Calcium (% DV),Iron (% DV),Caffeine (mg)
0,Coffee,Brewed Coffee,Short,3,0.1,0.0,0.0,0,5,0,0,0,0.3,0%,0%,0%,0%,175
1,Coffee,Brewed Coffee,Tall,4,0.1,0.0,0.0,0,10,0,0,0,0.5,0%,0%,0%,0%,260
2,Coffee,Brewed Coffee,Grande,5,0.1,0.0,0.0,0,10,0,0,0,1.0,0%,0%,0%,0%,330
3,Coffee,Brewed Coffee,Venti,5,0.1,0.0,0.0,0,10,0,0,0,1.0,0%,0%,2%,0%,410
4,Classic Espresso Drinks,Caffè Latte,Short Nonfat Milk,70,0.1,0.1,0.0,5,75,10,0,9,6.0,10%,0%,20%,0%,75
237,Frappuccino® Blended Crème,Strawberries & Crème (Without Whipped Cream),Soymilk,320,3 2,0.4,0.0,0,250,67,1,64,5.0,6%,8%,20%,10%,0
238,Frappuccino® Blended Crème,Vanilla Bean (Without Whipped Cream),Tall Nonfat Milk,170,0.1,0.1,0.0,0,160,39,0,38,4.0,6%,0%,10%,0%,0
239,Frappuccino® Blended Crème,Vanilla Bean (Without Whipped Cream),Whole Milk,200,3.5,2.0,0.1,10,160,39,0,38,3.0,6%,0%,10%,0%,0
240,Frappuccino® Blended Crème,Vanilla Bean (Without Whipped Cream),Soymilk,180,1.5,0.2,0.0,0,160,37,1,35,3.0,4%,0%,10%,6%,0
241,Frappuccino® Blended Crème,Vanilla Bean (Without Whipped Cream),Grande Nonfat Milk,240,0.1,0.1,0.0,5,230,56,0,55,5.0,8%,0%,15%,0%,0


In [15]:
df_starbucks_drinkMenu_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Beverage_category          242 non-null    object 
 1   Beverage                   242 non-null    object 
 2   Beverage_prep              242 non-null    object 
 3   Calories                   242 non-null    int64  
 4    Total Fat (g)             242 non-null    object 
 5   Trans Fat (g)              242 non-null    float64
 6   Saturated Fat (g)          242 non-null    float64
 7    Sodium (mg)               242 non-null    int64  
 8    Total Carbohydrates (g)   242 non-null    int64  
 9   Cholesterol (mg)           242 non-null    int64  
 10   Dietary Fibre (g)         242 non-null    int64  
 11   Sugars (g)                242 non-null    int64  
 12   Protein (g)               242 non-null    float64
 13  Vitamin A (% DV)           242 non-null    object 

In [10]:
df_starbucks_menu_nutrition_food2 = rcr('starbucks-menu-nutrition-food2.csv')
p(df_starbucks_menu_nutrition_food2)

(113, 6)


,Unnamed: 0,Calories,Fat (g),Carb. (g),Fiber (g),Protein (g)
0,Chonga Bagel,300,5.0,50,3,12
1,8-Grain Roll,380,6.0,70,7,10
2,Almond Croissant,410,22.0,45,3,10
3,Apple Fritter,460,23.0,56,2,7
4,Banana Nut Bread,420,22.0,52,2,6
108,Justin's Chocolate Hazelnut Butter,180,14.0,12,3,4
109,Justin's Classic Almond Butter,190,18.0,6,3,7
110,Lemon Chiffon Yogurt,340,13.0,38,0,18
111,Organic Avocado (Spread),90,8.0,5,4,1
112,Seasonal Fruit Blend,90,0.0,24,4,1


In [11]:
df_starbucks_menu_nutrition_drinks = pd.read_csv(os.path.join('..', 'raw_data','starbucks-menu-nutrition-drinks.csv'),na_values=["-"])
p(df_starbucks_menu_nutrition_drinks)

(177, 7)


,Unnamed: 0,Calories,Fat (g),Carb. (g),Fiber (g),Protein,Sodium
0,Cool Lime Starbucks Refreshers™ Beverage,45.0,0.0,11.0,0.0,0.0,10.0
1,Ombré Pink Drink,NaN,NaN,NaN,NaN,NaN,NaN
2,Pink Drink,NaN,NaN,NaN,NaN,NaN,NaN
3,Strawberry Acai Starbucks Refreshers™ Beverage,80.0,0.0,18.0,1.0,0.0,10.0
4,Very Berry Hibiscus Starbucks Refreshers™ Beverage,60.0,0.0,14.0,1.0,0.0,10.0
172,Chocolate Smoothie,320.0,5.0,53.0,8.0,20.0,170.0
173,Strawberry Smoothie,300.0,2.0,60.0,7.0,16.0,130.0
174,Ginger Ale,NaN,NaN,NaN,NaN,NaN,NaN
175,Lemon Ale,NaN,NaN,NaN,NaN,NaN,NaN
176,Orange Cream Soda,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_starbucks_menu_nutrition_drinks.set_index('Unnamed: 0', inplace=True)
p(df_starbucks_menu_nutrition_drinks)

(177, 6)


,Calories,Fat (g),Carb. (g),Fiber (g),Protein,Sodium
Unnamed: 0,,,,,,
Cool Lime Starbucks Refreshers™ Beverage,45.0,0.0,11.0,0.0,0.0,10.0
Ombré Pink Drink,NaN,NaN,NaN,NaN,NaN,NaN
Pink Drink,NaN,NaN,NaN,NaN,NaN,NaN
Strawberry Acai Starbucks Refreshers™ Beverage,80.0,0.0,18.0,1.0,0.0,10.0
Very Berry Hibiscus Starbucks Refreshers™ Beverage,60.0,0.0,14.0,1.0,0.0,10.0
Chocolate Smoothie,320.0,5.0,53.0,8.0,20.0,170.0
Strawberry Smoothie,300.0,2.0,60.0,7.0,16.0,130.0
Ginger Ale,NaN,NaN,NaN,NaN,NaN,NaN
Lemon Ale,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_starbucks_menu_nutrition_drinks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177 entries, Cool Lime Starbucks Refreshers™ Beverage to Orange Cream Soda
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Calories   92 non-null     float64
 1   Fat (g)    92 non-null     float64
 2   Carb. (g)  92 non-null     float64
 3   Fiber (g)  92 non-null     float64
 4   Protein    92 non-null     float64
 5   Sodium     92 non-null     float64
dtypes: float64(6)
memory usage: 9.7+ KB


In [ ]:
#df_starbucks_drinkMenu_expanded vs df_starbucks_menu_nutrition_drinks

In [16]:
p(df_starbucks_drinkMenu_expanded)

(242, 18)


,Beverage_category,Beverage,Beverage_prep,Calories,Total Fat (g),Trans Fat (g),Saturated Fat (g),Sodium (mg),Total Carbohydrates (g),Cholesterol (mg),Dietary Fibre (g),Sugars (g),Protein (g),Vitamin A (% DV),Vitamin C (% DV),Calcium (% DV),Iron (% DV),Caffeine (mg)
0,Coffee,Brewed Coffee,Short,3,0.1,0.0,0.0,0,5,0,0,0,0.3,0%,0%,0%,0%,175
1,Coffee,Brewed Coffee,Tall,4,0.1,0.0,0.0,0,10,0,0,0,0.5,0%,0%,0%,0%,260
2,Coffee,Brewed Coffee,Grande,5,0.1,0.0,0.0,0,10,0,0,0,1.0,0%,0%,0%,0%,330
3,Coffee,Brewed Coffee,Venti,5,0.1,0.0,0.0,0,10,0,0,0,1.0,0%,0%,2%,0%,410
4,Classic Espresso Drinks,Caffè Latte,Short Nonfat Milk,70,0.1,0.1,0.0,5,75,10,0,9,6.0,10%,0%,20%,0%,75
237,Frappuccino® Blended Crème,Strawberries & Crème (Without Whipped Cream),Soymilk,320,3 2,0.4,0.0,0,250,67,1,64,5.0,6%,8%,20%,10%,0
238,Frappuccino® Blended Crème,Vanilla Bean (Without Whipped Cream),Tall Nonfat Milk,170,0.1,0.1,0.0,0,160,39,0,38,4.0,6%,0%,10%,0%,0
239,Frappuccino® Blended Crème,Vanilla Bean (Without Whipped Cream),Whole Milk,200,3.5,2.0,0.1,10,160,39,0,38,3.0,6%,0%,10%,0%,0
240,Frappuccino® Blended Crème,Vanilla Bean (Without Whipped Cream),Soymilk,180,1.5,0.2,0.0,0,160,37,1,35,3.0,4%,0%,10%,6%,0
241,Frappuccino® Blended Crème,Vanilla Bean (Without Whipped Cream),Grande Nonfat Milk,240,0.1,0.1,0.0,5,230,56,0,55,5.0,8%,0%,15%,0%,0


In [20]:
df_starbucks_drinkMenu_expanded.loc[df_starbucks_drinkMenu_expanded.Beverage.str.contains('Lemon'), :]

,Beverage_category,Beverage,Beverage_prep,Calories,Total Fat (g),Trans Fat (g),Saturated Fat (g),Sodium (mg),Total Carbohydrates (g),Cholesterol (mg),Dietary Fibre (g),Sugars (g),Protein (g),Vitamin A (% DV),Vitamin C (% DV),Calcium (% DV),Iron (% DV),Caffeine (mg)
169,Shaken Iced Beverages,Shaken Iced Tazo® Tea Lemonade (With Classic Syrup),Tall,100,0,0.0,0.0,0,0,25,0,24,0.1,0%,10%,0%,0%,Varies
170,Shaken Iced Beverages,Shaken Iced Tazo® Tea Lemonade (With Classic Syrup),Grande,130,0,0.0,0.0,0,0,33,0,33,0.1,0%,15%,0%,0%,Varies
171,Shaken Iced Beverages,Shaken Iced Tazo® Tea Lemonade (With Classic Syrup),Venti,190,0,0.0,0.0,0,0,49,0,49,0.1,0%,20%,0%,0%,Varies


In [21]:
df_starbucks_menu_nutrition_drinks.loc[df_starbucks_menu_nutrition_drinks.index.str.contains('Lemon'), :]

,Calories,Fat (g),Carb. (g),Fiber (g),Protein,Sodium
Unnamed: 0,,,,,,
Tazo® Bottled Black with Lemon,140.0,0.0,35.0,0.0,0.0,10.0
Tazo® Bottled Lemon Ginger,120.0,0.0,31.0,0.0,0.0,10.0
Tazo® Bottled Organic Black Lemonade,140.0,0.0,35.0,0.0,0.0,10.0
Teavana® Shaken Iced Black Tea Lemonade,70.0,0.0,17.0,0.0,0.0,0.0
Teavana® Shaken Iced Green Tea Lemonade,70.0,0.0,17.0,0.0,0.0,5.0
Teavana® Shaken Iced Passion Tango™ Tea Lemonade,90.0,0.0,24.0,0.0,0.0,0.0
Starbucks Refreshers™ Strawberry Lemonade,90.0,0.0,27.0,0.0,0.0,0.0
Tazo® Bottled Black with Lemon,140.0,0.0,35.0,0.0,0.0,10.0
Tazo® Bottled Lemon Ginger,120.0,0.0,31.0,0.0,0.0,10.0


In [5]:
list(df_71C_merge_71A_71B_2010_2018.columns)

['game_id',
 'team_id',
 'index',
 'TEAM_NAME',
 'TEAM_ABBREVIATION',
 'TEAM_CITY',
 'MIN',
 'duration',
 'E_OFF_RATING',
 'OFF_RATING',
 'E_DEF_RATING',
 'DEF_RATING',
 'E_NET_RATING',
 'NET_RATING',
 'AST_PCT',
 'AST_TOV',
 'AST_RATIO',
 'OREB_PCT',
 'DREB_PCT',
 'REB_PCT',
 'E_TM_TOV_PCT',
 'TM_TOV_PCT',
 'EFG_PCT',
 'TS_PCT',
 'USG_PCT',
 'E_USG_PCT',
 'E_PACE',
 'PACE',
 'PACE_PER40',
 'POSS',
 'PIE',
 'duration_minutes',
 'w_pct',
 'min',
 'fgm',
 'fga',
 'fg_pct',
 'fg3m',
 'fg3a',
 'fg3_pct',
 'ftm',
 'fta',
 'ft_pct',
 'oreb',
 'dreb',
 'reb',
 'ast',
 'stl',
 'blk',
 'tov',
 'pf',
 'pts',
 'wl',
 'spread',
 'away_home_',
 'game_date',
 'matchup',
 'season_type',
 'season',
 'season_year',
 'latitude',
 'longitude',
 'game_latitude',
 'game_longitude',
 'travel_distance',
 'game_time_zone',
 'time_zone',
 'time_zone_diff',
 'time_zone_diff_abs',
 'city_proper_metro_area',
 'Country/Region',
 'city_proper_metro_area_GDP_bil',
 'city_proper_metro_area_GDP_bil1',
 'game_date_diff

In [6]:
column_drop_list = ['index',
 'TEAM_NAME',
 'TEAM_ABBREVIATION',
 'TEAM_CITY',
 'MIN',
 'duration',
 'E_OFF_RATING',
 'OFF_RATING',
 'E_DEF_RATING',
 'DEF_RATING',
 'E_NET_RATING',
 'NET_RATING',
 'AST_PCT',
 'AST_TOV',
 'AST_RATIO',
 'OREB_PCT',
 'DREB_PCT',
 'REB_PCT',
 'E_TM_TOV_PCT',
 'TM_TOV_PCT',
 'EFG_PCT',
 'TS_PCT',
 'USG_PCT',
 'E_USG_PCT',
 'E_PACE',
 'PACE',
 'PACE_PER40',
 'POSS',
 'PIE',
 'duration_minutes',
 'w_pct',
 'min',
 'fgm',
 'fga',
 'fg_pct',
 'fg3m',
 'fg3a',
 'fg3_pct',
 'ftm',
 'fta',
 'ft_pct',
 'oreb',
 'dreb',
 'reb',
 'ast',
 'stl',
 'blk',
 'tov',
 'pf',
 'pts',
 'wl',
 'spread',                    
 'matchup',
 'season_type',
 'season_year',
 '_merge']
pl(column_drop_list)

56


['index',
 'TEAM_NAME',
 'TEAM_ABBREVIATION',
 'TEAM_CITY',
 'MIN',
 'duration',
 'E_OFF_RATING',
 'OFF_RATING',
 'E_DEF_RATING',
 'DEF_RATING',
 'E_NET_RATING',
 'NET_RATING',
 'AST_PCT',
 'AST_TOV',
 'AST_RATIO',
 'OREB_PCT',
 'DREB_PCT',
 'REB_PCT',
 'E_TM_TOV_PCT',
 'TM_TOV_PCT',
 'EFG_PCT',
 'TS_PCT',
 'USG_PCT',
 'E_USG_PCT',
 'E_PACE',
 'PACE',
 'PACE_PER40',
 'POSS',
 'PIE',
 'duration_minutes',
 'w_pct',
 'min',
 'fgm',
 'fga',
 'fg_pct',
 'fg3m',
 'fg3a',
 'fg3_pct',
 'ftm',
 'fta',
 'ft_pct',
 'oreb',
 'dreb',
 'reb',
 'ast',
 'stl',
 'blk',
 'tov',
 'pf',
 'pts',
 'wl',
 'spread',
 'matchup',
 'season_type',
 'season_year',
 '_merge']

In [7]:
df_71C_merge_71A_71B_2010_2018.drop(columns=column_drop_list, inplace=True)

In [8]:
p(df_71C_merge_71A_71B_2010_2018)

(20532, 78)


,game_id,team_id,away_home_,game_date,season,latitude,longitude,game_latitude,game_longitude,travel_distance,game_time_zone,time_zone,time_zone_diff,time_zone_diff_abs,city_proper_metro_area,Country/Region,city_proper_metro_area_GDP_bil,city_proper_metro_area_GDP_bil1,game_date_diff,game_date_diff1,games_back_to_back_count_game_window2,games_back_to_back_count_game_window5,games_back_to_back_count_game_window12,games_back_to_back_count_game_window999,games_back_to_back_percentage_game_window2,games_back_to_back_percentage_game_window5,games_back_to_back_percentage_game_window12,games_back_to_back_percentage_game_window999,away_game_count_day_window2,home_game_count_day_window2,game_count_day_window2,away_game_percent_day_window2,home_game_percent_day_window2,away_game_count_day_window5,home_game_count_day_window5,game_count_day_window5,away_game_percent_day_window5,home_game_percent_day_window5,away_game_count_day_window8,home_game_count_day_window8,game_count_day_window8,away_game_percent_day_window8,home_game_percent_day_window8,away_game_count_day_window12,home_game_count_day_window12,game_count_day_window12,away_game_percent_day_window12,home_game_percent_day_window12,away_game_count_day_window999,home_game_count_day_window999,game_count_day_window999,away_game_percent_day_window999,home_game_percent_day_window999,future_away_game_count_day_window2,future_home_game_count_day_window2,future_game_count_day_window2,future_away_game_percent_day_window2,future_home_game_percent_day_window2,future_away_game_count_day_window5,future_home_game_count_day_window5,future_game_count_day_window5,future_away_game_percent_day_window5,future_home_game_percent_day_window5,future_away_game_count_day_window8,future_home_game_count_day_window8,future_game_count_day_window8,future_away_game_percent_day_window8,future_home_game_percent_day_window8,future_away_game_count_day_window12,future_home_game_count_day_window12,future_game_count_day_window12,future_away_game_percent_day_window12,future_home_game_percent_day_window12,future_away_game_count_day_window999,future_home_game_count_day_window999,future_game_count_day_window999,future_away_game_percent_day_window999,future_home_game_percent_day_window999
0,21000001,1610612748,0,2010-10-26,2010-11,25.774173,-80.193620,42.360253,-71.058291,1257.755091,America/New_York,America/New_York,0,0,Miami,United States,377.500,262.7,9,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.000000,0.000000,1.0,0.0,1.0,1.00,0.00,1.0,0.0,1.0,1.000000,0.000000,2.0,0.0,2.0,1.0,0.0,2.0,1.0,3.0,0.666667,0.333333,3.0,2.0,5.0,0.600000,0.400000,4.0,3.0,7.0,0.571429,0.428571,41.0,41.0,82.0,0.5,0.5
1,21000001,1610612738,1,2010-10-26,2010-11,42.360253,-71.058291,42.360253,-71.058291,0.000000,America/New_York,America/New_York,0,0,Boston,United States,463.570,560.1,9,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.000000,1.000000,0.0,1.0,1.0,0.00,1.00,0.0,1.0,1.0,0.000000,1.000000,1.0,1.0,2.0,0.5,0.5,1.0,2.0,3.0,0.333333,0.666667,2.0,2.0,4.0,0.500000,0.500000,2.0,4.0,6.0,0.333333,0.666667,41.0,41.0,82.0,0.5,0.5
2,21000002,1610612756,0,2010-10-26,2010-11,33.448437,-112.074142,45.520247,-122.674195,1005.606855,America/Los_Angeles,America/Phoenix,0,0,Phoenix,United States,255.211,607.1,9,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.000000,0.000000,1.0,0.0,1.0,1.00,0.00,1.0,0.0,1.0,1.000000,0.000000,1.0,0.0,1.0,1.0,0.0,2.0,1.0,3.0,0.666667,0.333333,2.0,1.0,3.0,0.666667,0.333333,2.0,3.0,5.0,0.400000,0.600000,41.0,41.0,82.0,0.5,0.5
3,21000002,1610612757,1,2010-10-26,2010-11,45.520247,-122.674195,45.520247,-122.674195,0.000000,America/Los_Angeles,America/Los_Angeles,0,0,Portland,United States,164.419,158.5,9,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.000000,1.000000,0.0,1.0,1.0,0.00,1.00,0.0,1.0,1.0,0.000000,1.000000,1.0,1.0,2.0,0.5,0.5,2.0,1.0,3.0,0.66

In [9]:
df_71C_merge_71A_71B_2010_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20532 entries, 0 to 20531
Data columns (total 78 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   game_id                                       20532 non-null  int64  
 1   team_id                                       20532 non-null  int64  
 2   away_home_                                    20532 non-null  int64  
 3   game_date                                     20532 non-null  object 
 4   season                                        20532 non-null  object 
 5   latitude                                      20532 non-null  float64
 6   longitude                                     20532 non-null  float64
 7   game_latitude                                 20532 non-null  float64
 8   game_longitude                                20532 non-null  float64
 9   travel_distance                               20532 non-null 

In [10]:
help(f.get_numeric_diff)

Help on function get_numeric_diff in module my_functions:

get_numeric_diff(df_, groupby_list_, not_difference_list_, suffix_list_)



In [11]:
help(f.tall2wide2)

Help on function tall2wide2 in module my_functions:

tall2wide2(df_, not_first_second_list_, indicator_column_, keep_indicator_=False, first_second_column_suffix_=None)



In [12]:
p(df_71C_merge_71A_71B_2010_2018)

(20532, 78)


,game_id,team_id,away_home_,game_date,season,latitude,longitude,game_latitude,game_longitude,travel_distance,game_time_zone,time_zone,time_zone_diff,time_zone_diff_abs,city_proper_metro_area,Country/Region,city_proper_metro_area_GDP_bil,city_proper_metro_area_GDP_bil1,game_date_diff,game_date_diff1,games_back_to_back_count_game_window2,games_back_to_back_count_game_window5,games_back_to_back_count_game_window12,games_back_to_back_count_game_window999,games_back_to_back_percentage_game_window2,games_back_to_back_percentage_game_window5,games_back_to_back_percentage_game_window12,games_back_to_back_percentage_game_window999,away_game_count_day_window2,home_game_count_day_window2,game_count_day_window2,away_game_percent_day_window2,home_game_percent_day_window2,away_game_count_day_window5,home_game_count_day_window5,game_count_day_window5,away_game_percent_day_window5,home_game_percent_day_window5,away_game_count_day_window8,home_game_count_day_window8,game_count_day_window8,away_game_percent_day_window8,home_game_percent_day_window8,away_game_count_day_window12,home_game_count_day_window12,game_count_day_window12,away_game_percent_day_window12,home_game_percent_day_window12,away_game_count_day_window999,home_game_count_day_window999,game_count_day_window999,away_game_percent_day_window999,home_game_percent_day_window999,future_away_game_count_day_window2,future_home_game_count_day_window2,future_game_count_day_window2,future_away_game_percent_day_window2,future_home_game_percent_day_window2,future_away_game_count_day_window5,future_home_game_count_day_window5,future_game_count_day_window5,future_away_game_percent_day_window5,future_home_game_percent_day_window5,future_away_game_count_day_window8,future_home_game_count_day_window8,future_game_count_day_window8,future_away_game_percent_day_window8,future_home_game_percent_day_window8,future_away_game_count_day_window12,future_home_game_count_day_window12,future_game_count_day_window12,future_away_game_percent_day_window12,future_home_game_percent_day_window12,future_away_game_count_day_window999,future_home_game_count_day_window999,future_game_count_day_window999,future_away_game_percent_day_window999,future_home_game_percent_day_window999
0,21000001,1610612748,0,2010-10-26,2010-11,25.774173,-80.193620,42.360253,-71.058291,1257.755091,America/New_York,America/New_York,0,0,Miami,United States,377.500,262.7,9,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.000000,0.000000,1.0,0.0,1.0,1.00,0.00,1.0,0.0,1.0,1.000000,0.000000,2.0,0.0,2.0,1.0,0.0,2.0,1.0,3.0,0.666667,0.333333,3.0,2.0,5.0,0.600000,0.400000,4.0,3.0,7.0,0.571429,0.428571,41.0,41.0,82.0,0.5,0.5
1,21000001,1610612738,1,2010-10-26,2010-11,42.360253,-71.058291,42.360253,-71.058291,0.000000,America/New_York,America/New_York,0,0,Boston,United States,463.570,560.1,9,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.000000,1.000000,0.0,1.0,1.0,0.00,1.00,0.0,1.0,1.0,0.000000,1.000000,1.0,1.0,2.0,0.5,0.5,1.0,2.0,3.0,0.333333,0.666667,2.0,2.0,4.0,0.500000,0.500000,2.0,4.0,6.0,0.333333,0.666667,41.0,41.0,82.0,0.5,0.5
2,21000002,1610612756,0,2010-10-26,2010-11,33.448437,-112.074142,45.520247,-122.674195,1005.606855,America/Los_Angeles,America/Phoenix,0,0,Phoenix,United States,255.211,607.1,9,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.000000,0.000000,1.0,0.0,1.0,1.00,0.00,1.0,0.0,1.0,1.000000,0.000000,1.0,0.0,1.0,1.0,0.0,2.0,1.0,3.0,0.666667,0.333333,2.0,1.0,3.0,0.666667,0.333333,2.0,3.0,5.0,0.400000,0.600000,41.0,41.0,82.0,0.5,0.5
3,21000002,1610612757,1,2010-10-26,2010-11,45.520247,-122.674195,45.520247,-122.674195,0.000000,America/Los_Angeles,America/Los_Angeles,0,0,Portland,United States,164.419,158.5,9,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.000000,1.000000,0.0,1.0,1.0,0.00,1.00,0.0,1.0,1.0,0.000000,1.000000,1.0,1.0,2.0,0.5,0.5,2.0,1.0,3.0,0.66

In [13]:
df_56game_id_team_id_a_b_matchup_a_b_2010_2018 = \
rcp('56game_id_team_id_a_b_matchup_a_b_2010_2018.csv')

p(df_56game_id_team_id_a_b_matchup_a_b_2010_2018)

(20532, 17)


,game_id,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,team_id,w_pct,wl,away_home_,game_date,matchup,season_type,season,season_year,a,b,_a,_b
0,21000001,Heat,MIA,Miami,1610612748,0.0,L,0,2010-10-26,BOS vs. MIA,Regular Season,2010-11,2010,BOS,MIA,0.0,1.0
1,21000001,Celtics,BOS,Boston,1610612738,1.0,W,1,2010-10-26,BOS vs. MIA,Regular Season,2010-11,2010,BOS,MIA,1.0,0.0
2,21000002,Suns,PHX,Phoenix,1610612756,0.0,L,0,2010-10-26,PHX vs. POR,Regular Season,2010-11,2010,PHX,POR,1.0,0.0
3,21000002,Trail Blazers,POR,Portland,1610612757,1.0,W,1,2010-10-26,PHX vs. POR,Regular Season,2010-11,2010,PHX,POR,0.0,1.0
4,21000003,Rockets,HOU,Houston,1610612745,0.0,L,0,2010-10-26,HOU vs. LAL,Regular Season,2010-11,2010,HOU,LAL,1.0,0.0
20527,41700402,Warriors,GSW,Golden State,1610612744,1.0,W,1,2018-06-03,CLE vs. GSW,Playoffs,2017-18,2017,CLE,GSW,0.0,1.0
20528,41700403,Warriors,GSW,Golden State,1610612744,1.0,W,0,2018-06-06,CLE vs. GSW,Playoffs,2017-18,2017,CLE,GSW,0.0,1.0
20529,41700403,Cavaliers,CLE,Cleveland,1610612739,0.0,L,1,2018-06-06,CLE vs. GSW,Playoffs,2017-18,2017,CLE,GSW,1.0,0.0
20530,41700404,Warriors,GSW,Golden State,1610612744,1.0,W,0,2018-06-08,CLE vs. GSW,Playoffs,2017-18,2017,CLE,GSW,0.0,1.0
20531,41700404,Cavaliers,CLE,Cleveland,1610612739,0.0,L,1,2018-06-08,CLE vs. GSW,Playoffs,2017-18,2017,CLE,GSW,1.0,0.0


In [14]:
df_71C_merge_71A_71B_2010_2018 = pd.merge(df_56game_id_team_id_a_b_matchup_a_b_2010_2018.\
                                          loc[:, ['game_id', 'team_id', '_a']],
                                          df_71C_merge_71A_71B_2010_2018,
                                          on=['game_id', 'team_id'])
p(df_71C_merge_71A_71B_2010_2018)

(20532, 79)


,game_id,team_id,_a,away_home_,game_date,season,latitude,longitude,game_latitude,game_longitude,travel_distance,game_time_zone,time_zone,time_zone_diff,time_zone_diff_abs,city_proper_metro_area,Country/Region,city_proper_metro_area_GDP_bil,city_proper_metro_area_GDP_bil1,game_date_diff,game_date_diff1,games_back_to_back_count_game_window2,games_back_to_back_count_game_window5,games_back_to_back_count_game_window12,games_back_to_back_count_game_window999,games_back_to_back_percentage_game_window2,games_back_to_back_percentage_game_window5,games_back_to_back_percentage_game_window12,games_back_to_back_percentage_game_window999,away_game_count_day_window2,home_game_count_day_window2,game_count_day_window2,away_game_percent_day_window2,home_game_percent_day_window2,away_game_count_day_window5,home_game_count_day_window5,game_count_day_window5,away_game_percent_day_window5,home_game_percent_day_window5,away_game_count_day_window8,home_game_count_day_window8,game_count_day_window8,away_game_percent_day_window8,home_game_percent_day_window8,away_game_count_day_window12,home_game_count_day_window12,game_count_day_window12,away_game_percent_day_window12,home_game_percent_day_window12,away_game_count_day_window999,home_game_count_day_window999,game_count_day_window999,away_game_percent_day_window999,home_game_percent_day_window999,future_away_game_count_day_window2,future_home_game_count_day_window2,future_game_count_day_window2,future_away_game_percent_day_window2,future_home_game_percent_day_window2,future_away_game_count_day_window5,future_home_game_count_day_window5,future_game_count_day_window5,future_away_game_percent_day_window5,future_home_game_percent_day_window5,future_away_game_count_day_window8,future_home_game_count_day_window8,future_game_count_day_window8,future_away_game_percent_day_window8,future_home_game_percent_day_window8,future_away_game_count_day_window12,future_home_game_count_day_window12,future_game_count_day_window12,future_away_game_percent_day_window12,future_home_game_percent_day_window12,future_away_game_count_day_window999,future_home_game_count_day_window999,future_game_count_day_window999,future_away_game_percent_day_window999,future_home_game_percent_day_window999
0,21000001,1610612748,0.0,0,2010-10-26,2010-11,25.774173,-80.193620,42.360253,-71.058291,1257.755091,America/New_York,America/New_York,0,0,Miami,United States,377.500,262.7,9,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.000000,0.000000,1.0,0.0,1.0,1.00,0.00,1.0,0.0,1.0,1.000000,0.000000,2.0,0.0,2.0,1.0,0.0,2.0,1.0,3.0,0.666667,0.333333,3.0,2.0,5.0,0.600000,0.400000,4.0,3.0,7.0,0.571429,0.428571,41.0,41.0,82.0,0.5,0.5
1,21000001,1610612738,1.0,1,2010-10-26,2010-11,42.360253,-71.058291,42.360253,-71.058291,0.000000,America/New_York,America/New_York,0,0,Boston,United States,463.570,560.1,9,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.000000,1.000000,0.0,1.0,1.0,0.00,1.00,0.0,1.0,1.0,0.000000,1.000000,1.0,1.0,2.0,0.5,0.5,1.0,2.0,3.0,0.333333,0.666667,2.0,2.0,4.0,0.500000,0.500000,2.0,4.0,6.0,0.333333,0.666667,41.0,41.0,82.0,0.5,0.5
2,21000002,1610612756,1.0,0,2010-10-26,2010-11,33.448437,-112.074142,45.520247,-122.674195,1005.606855,America/Los_Angeles,America/Phoenix,0,0,Phoenix,United States,255.211,607.1,9,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.000000,0.000000,1.0,0.0,1.0,1.00,0.00,1.0,0.0,1.0,1.000000,0.000000,1.0,0.0,1.0,1.0,0.0,2.0,1.0,3.0,0.666667,0.333333,2.0,1.0,3.0,0.666667,0.333333,2.0,3.0,5.0,0.400000,0.600000,41.0,41.0,82.0,0.5,0.5
3,21000002,1610612757,0.0,1,2010-10-26,2010-11,45.520247,-122.674195,45.520247,-122.674195,0.000000,America/Los_Angeles,America/Los_Angeles,0,0,Portland,United States,164.419,158.5,9,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.000000,1.000000,0.0,1.0,1.0,0.00,1.00,0.0,1.0,1.0,0.000000,1.000000,1.0,1.0,2.0,0.5,0

In [15]:
df_71C_merge_71A_71B_2010_2018.loc[:, '_a'] = \
abs(df_71C_merge_71A_71B_2010_2018.loc[:, '_a'] - 1)

p(df_71C_merge_71A_71B_2010_2018.loc[:, '_a'])

(20532,)


0        1.0
1        0.0
2        0.0
3        1.0
4        0.0
20527    1.0
20528    1.0
20529    0.0
20530    1.0
20531    0.0
Name: _a, dtype: float64

In [16]:
###########give me arguments please
df_71C_merge_71A_71B_2010_2018_wide = \
f.tall2wide2(df_71C_merge_71A_71B_2010_2018,
             ['game_id', 'game_date', 'season'],
             '_a')

p(df_71C_merge_71A_71B_2010_2018_wide)

(10266, 153)


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,game_id,team_id_a,away_home__a,game_date,season,latitude_a,longitude_a,game_latitude_a,game_longitude_a,travel_distance_a,game_time_zone_a,time_zone_a,time_zone_diff_a,time_zone_diff_abs_a,city_proper_metro_area_a,Country/Region_a,city_proper_metro_area_GDP_bil_a,city_proper_metro_area_GDP_bil1_a,game_date_diff_a,game_date_diff1_a,games_back_to_back_count_game_window2_a,games_back_to_back_count_game_window5_a,games_back_to_back_count_game_window12_a,games_back_to_back_count_game_window999_a,games_back_to_back_percentage_game_window2_a,games_back_to_back_percentage_game_window5_a,games_back_to_back_percentage_game_window12_a,games_back_to_back_percentage_game_window999_a,away_game_count_day_window2_a,home_game_count_day_window2_a,game_count_day_window2_a,away_game_percent_day_window2_a,home_game_percent_day_window2_a,away_game_count_day_window5_a,home_game_count_day_window5_a,game_count_day_window5_a,away_game_percent_day_window5_a,home_game_percent_day_window5_a,away_game_count_day_window8_a,home_game_count_day_window8_a,game_count_day_window8_a,away_game_percent_day_window8_a,home_game_percent_day_window8_a,away_game_count_day_window12_a,home_game_count_day_window12_a,game_count_day_window12_a,away_game_percent_day_window12_a,home_game_percent_day_window12_a,away_game_count_day_window999_a,home_game_count_day_window999_a,game_count_day_window999_a,away_game_percent_day_window999_a,home_game_percent_day_window999_a,future_away_game_count_day_window2_a,future_home_game_count_day_window2_a,future_game_count_day_window2_a,future_away_game_percent_day_window2_a,future_home_game_percent_day_window2_a,future_away_game_count_day_window5_a,future_home_game_count_day_window5_a,future_game_count_day_window5_a,future_away_game_percent_day_window5_a,future_home_game_percent_day_window5_a,future_away_game_count_day_window8_a,future_home_game_count_day_window8_a,future_game_count_day_window8_a,future_away_game_percent_day_window8_a,future_home_game_percent_day_window8_a,future_away_game_count_day_window12_a,future_home_game_count_day_window12_a,future_game_count_day_window12_a,future_away_game_percent_day_window12_a,future_home_game_percent_day_window12_a,future_away_game_count_day_window999_a,future_home_game_count_day_window999_a,future_game_count_day_window999_a,future_away_game_percent_day_window999_a,future_home_game_percent_day_window999_a,team_id_b,away_home__b,latitude_b,longitude_b,game_latitude_b,game_longitude_b,travel_distance_b,game_time_zone_b,time_zone_b,time_zone_diff_b,time_zone_diff_abs_b,city_proper_metro_area_b,Country/Region_b,city_proper_metro_area_GDP_bil_b,city_proper_metro_area_GDP_bil1_b,game_date_diff_b,game_date_diff1_b,games_back_to_back_count_game_window2_b,games_back_to_back_count_game_window5_b,games_back_to_back_count_game_window12_b,games_back_to_back_count_game_window999_b,games_back_to_back_percentage_game_window2_b,games_back_to_back_percentage_game_window5_b,games_back_to_back_percentage_game_window12_b,games_back_to_back_percentage_game_window999_b,away_game_count_day_window2_b,home_game_count_day_window2_b,game_count_day_window2_b,away_game_percent_day_window2_b,home_game_percent_day_window2_b,away_game_count_day_window5_b,home_game_count_day_window5_b,game_count_day_window5_b,away_game_percent_day_window5_b,home_game_percent_day_window5_b,away_game_count_day_window8_b,home_game_count_day_window8_b,game_count_day_window8_b,away_game_percent_day_window8_b,home_game_percent_day_window8_b,away_game_count_day_window12_b,home_game_count_day_window12_b,game_count_day_window12_b,away_game_percent_day_window12_b,home_game_percent_day_window12_b,away_game_count_day_window999_b,home_game_count_day_window999_b,game_count_day_window999_b,away_game_percent_day_window999_b,home_game_percent_day_window999_b,future_away_game_count_day_window2_b,future_home_game_count_day_window2_b,future_game_count_day_window2_b,future_away_game_percent_day_window2_b,future_home_game_percent_day_window2_b,future_away_game_count_day_

In [17]:
column_underscore_area_list = \
[k for k in df_71C_merge_71A_71B_2010_2018_wide.columns if '_area' in k]

pl(column_underscore_area_list)

6


['city_proper_metro_area_a',
 'city_proper_metro_area_GDP_bil_a',
 'city_proper_metro_area_GDP_bil1_a',
 'city_proper_metro_area_b',
 'city_proper_metro_area_GDP_bil_b',
 'city_proper_metro_area_GDP_bil1_b']

In [18]:
column_underscore_Area_list = \
[k.split('_area')[0] + '_Area' + k.split('_area')[1] for k in column_underscore_area_list]

pl(column_underscore_Area_list)

6


['city_proper_metro_Area_a',
 'city_proper_metro_Area_GDP_bil_a',
 'city_proper_metro_Area_GDP_bil1_a',
 'city_proper_metro_Area_b',
 'city_proper_metro_Area_GDP_bil_b',
 'city_proper_metro_Area_GDP_bil1_b']

In [19]:
column_underscore_Area_dict = dict(zip(column_underscore_area_list, column_underscore_Area_list))
pdc(column_underscore_Area_dict)

6


{'city_proper_metro_area_a': 'city_proper_metro_Area_a',
 'city_proper_metro_area_GDP_bil_a': 'city_proper_metro_Area_GDP_bil_a',
 'city_proper_metro_area_GDP_bil1_a': 'city_proper_metro_Area_GDP_bil1_a',
 'city_proper_metro_area_b': 'city_proper_metro_Area_b',
 'city_proper_metro_area_GDP_bil_b': 'city_proper_metro_Area_GDP_bil_b',
 'city_proper_metro_area_GDP_bil1_b': 'city_proper_metro_Area_GDP_bil1_b'}

In [20]:
df_71C_merge_71A_71B_2010_2018_wide.rename(columns=column_underscore_Area_dict, inplace=True)
p(df_71C_merge_71A_71B_2010_2018_wide)

(10266, 153)


,game_id,team_id_a,away_home__a,game_date,season,latitude_a,longitude_a,game_latitude_a,game_longitude_a,travel_distance_a,game_time_zone_a,time_zone_a,time_zone_diff_a,time_zone_diff_abs_a,city_proper_metro_Area_a,Country/Region_a,city_proper_metro_Area_GDP_bil_a,city_proper_metro_Area_GDP_bil1_a,game_date_diff_a,game_date_diff1_a,games_back_to_back_count_game_window2_a,games_back_to_back_count_game_window5_a,games_back_to_back_count_game_window12_a,games_back_to_back_count_game_window999_a,games_back_to_back_percentage_game_window2_a,games_back_to_back_percentage_game_window5_a,games_back_to_back_percentage_game_window12_a,games_back_to_back_percentage_game_window999_a,away_game_count_day_window2_a,home_game_count_day_window2_a,game_count_day_window2_a,away_game_percent_day_window2_a,home_game_percent_day_window2_a,away_game_count_day_window5_a,home_game_count_day_window5_a,game_count_day_window5_a,away_game_percent_day_window5_a,home_game_percent_day_window5_a,away_game_count_day_window8_a,home_game_count_day_window8_a,game_count_day_window8_a,away_game_percent_day_window8_a,home_game_percent_day_window8_a,away_game_count_day_window12_a,home_game_count_day_window12_a,game_count_day_window12_a,away_game_percent_day_window12_a,home_game_percent_day_window12_a,away_game_count_day_window999_a,home_game_count_day_window999_a,game_count_day_window999_a,away_game_percent_day_window999_a,home_game_percent_day_window999_a,future_away_game_count_day_window2_a,future_home_game_count_day_window2_a,future_game_count_day_window2_a,future_away_game_percent_day_window2_a,future_home_game_percent_day_window2_a,future_away_game_count_day_window5_a,future_home_game_count_day_window5_a,future_game_count_day_window5_a,future_away_game_percent_day_window5_a,future_home_game_percent_day_window5_a,future_away_game_count_day_window8_a,future_home_game_count_day_window8_a,future_game_count_day_window8_a,future_away_game_percent_day_window8_a,future_home_game_percent_day_window8_a,future_away_game_count_day_window12_a,future_home_game_count_day_window12_a,future_game_count_day_window12_a,future_away_game_percent_day_window12_a,future_home_game_percent_day_window12_a,future_away_game_count_day_window999_a,future_home_game_count_day_window999_a,future_game_count_day_window999_a,future_away_game_percent_day_window999_a,future_home_game_percent_day_window999_a,team_id_b,away_home__b,latitude_b,longitude_b,game_latitude_b,game_longitude_b,travel_distance_b,game_time_zone_b,time_zone_b,time_zone_diff_b,time_zone_diff_abs_b,city_proper_metro_Area_b,Country/Region_b,city_proper_metro_Area_GDP_bil_b,city_proper_metro_Area_GDP_bil1_b,game_date_diff_b,game_date_diff1_b,games_back_to_back_count_game_window2_b,games_back_to_back_count_game_window5_b,games_back_to_back_count_game_window12_b,games_back_to_back_count_game_window999_b,games_back_to_back_percentage_game_window2_b,games_back_to_back_percentage_game_window5_b,games_back_to_back_percentage_game_window12_b,games_back_to_back_percentage_game_window999_b,away_game_count_day_window2_b,home_game_count_day_window2_b,game_count_day_window2_b,away_game_percent_day_window2_b,home_game_percent_day_window2_b,away_game_count_day_window5_b,home_game_count_day_window5_b,game_count_day_window5_b,away_game_percent_day_window5_b,home_game_percent_day_window5_b,away_game_count_day_window8_b,home_game_count_day_window8_b,game_count_day_window8_b,away_game_percent_day_window8_b,home_game_percent_day_window8_b,away_game_count_day_window12_b,home_game_count_day_window12_b,game_count_day_window12_b,away_game_percent_day_window12_b,home_game_percent_day_window12_b,away_game_count_day_window999_b,home_game_count_day_window999_b,game_count_day_window999_b,away_game_percent_day_window999_b,home_game_percent_day_window999_b,future_away_game_count_day_window2_b,future_home_game_count_day_window2_b,future_game_count_day_window2_b,future_away_game_percent_day_window2_b,future_home_game_percent_day_window2_b,future_away_game_count_day_

In [21]:
column_underscore_away_list = \
[k for k in df_71C_merge_71A_71B_2010_2018_wide.columns if '_away' in k]

pl(column_underscore_away_list)

20


['future_away_game_count_day_window2_a',
 'future_away_game_percent_day_window2_a',
 'future_away_game_count_day_window5_a',
 'future_away_game_percent_day_window5_a',
 'future_away_game_count_day_window8_a',
 'future_away_game_percent_day_window8_a',
 'future_away_game_count_day_window12_a',
 'future_away_game_percent_day_window12_a',
 'future_away_game_count_day_window999_a',
 'future_away_game_percent_day_window999_a',
 'future_away_game_count_day_window2_b',
 'future_away_game_percent_day_window2_b',
 'future_away_game_count_day_window5_b',
 'future_away_game_percent_day_window5_b',
 'future_away_game_count_day_window8_b',
 'future_away_game_percent_day_window8_b',
 'future_away_game_count_day_window12_b',
 'future_away_game_percent_day_window12_b',
 'future_away_game_count_day_window999_b',
 'future_away_game_percent_day_window999_b']

In [22]:
column_underscore_Away_list = \
[k.split('_away')[0] + '_Away' + k.split('_away')[1] for k in column_underscore_away_list]

pl(column_underscore_Away_list)

20


['future_Away_game_count_day_window2_a',
 'future_Away_game_percent_day_window2_a',
 'future_Away_game_count_day_window5_a',
 'future_Away_game_percent_day_window5_a',
 'future_Away_game_count_day_window8_a',
 'future_Away_game_percent_day_window8_a',
 'future_Away_game_count_day_window12_a',
 'future_Away_game_percent_day_window12_a',
 'future_Away_game_count_day_window999_a',
 'future_Away_game_percent_day_window999_a',
 'future_Away_game_count_day_window2_b',
 'future_Away_game_percent_day_window2_b',
 'future_Away_game_count_day_window5_b',
 'future_Away_game_percent_day_window5_b',
 'future_Away_game_count_day_window8_b',
 'future_Away_game_percent_day_window8_b',
 'future_Away_game_count_day_window12_b',
 'future_Away_game_percent_day_window12_b',
 'future_Away_game_count_day_window999_b',
 'future_Away_game_percent_day_window999_b']

In [23]:
column_underscore_away_Away_dict = dict(zip(column_underscore_away_list, column_underscore_Away_list))
pdc(column_underscore_away_Away_dict)

20


{'future_away_game_count_day_window2_a': 'future_Away_game_count_day_window2_a',
 'future_away_game_percent_day_window2_a': 'future_Away_game_percent_day_window2_a',
 'future_away_game_count_day_window5_a': 'future_Away_game_count_day_window5_a',
 'future_away_game_percent_day_window5_a': 'future_Away_game_percent_day_window5_a',
 'future_away_game_count_day_window8_a': 'future_Away_game_count_day_window8_a',
 'future_away_game_percent_day_window8_a': 'future_Away_game_percent_day_window8_a',
 'future_away_game_count_day_window12_a': 'future_Away_game_count_day_window12_a',
 'future_away_game_percent_day_window12_a': 'future_Away_game_percent_day_window12_a',
 'future_away_game_count_day_window999_a': 'future_Away_game_count_day_window999_a',
 'future_away_game_percent_day_window999_a': 'future_Away_game_percent_day_window999_a',
 'future_away_game_count_day_window2_b': 'future_Away_game_count_day_window2_b',
 'future_away_game_percent_day_window2_b': 'future_Away_game_percent_day_wind

In [24]:
df_71C_merge_71A_71B_2010_2018_wide.rename(columns=column_underscore_away_Away_dict, inplace=True)
p(df_71C_merge_71A_71B_2010_2018_wide)

(10266, 153)


,game_id,team_id_a,away_home__a,game_date,season,latitude_a,longitude_a,game_latitude_a,game_longitude_a,travel_distance_a,game_time_zone_a,time_zone_a,time_zone_diff_a,time_zone_diff_abs_a,city_proper_metro_Area_a,Country/Region_a,city_proper_metro_Area_GDP_bil_a,city_proper_metro_Area_GDP_bil1_a,game_date_diff_a,game_date_diff1_a,games_back_to_back_count_game_window2_a,games_back_to_back_count_game_window5_a,games_back_to_back_count_game_window12_a,games_back_to_back_count_game_window999_a,games_back_to_back_percentage_game_window2_a,games_back_to_back_percentage_game_window5_a,games_back_to_back_percentage_game_window12_a,games_back_to_back_percentage_game_window999_a,away_game_count_day_window2_a,home_game_count_day_window2_a,game_count_day_window2_a,away_game_percent_day_window2_a,home_game_percent_day_window2_a,away_game_count_day_window5_a,home_game_count_day_window5_a,game_count_day_window5_a,away_game_percent_day_window5_a,home_game_percent_day_window5_a,away_game_count_day_window8_a,home_game_count_day_window8_a,game_count_day_window8_a,away_game_percent_day_window8_a,home_game_percent_day_window8_a,away_game_count_day_window12_a,home_game_count_day_window12_a,game_count_day_window12_a,away_game_percent_day_window12_a,home_game_percent_day_window12_a,away_game_count_day_window999_a,home_game_count_day_window999_a,game_count_day_window999_a,away_game_percent_day_window999_a,home_game_percent_day_window999_a,future_Away_game_count_day_window2_a,future_home_game_count_day_window2_a,future_game_count_day_window2_a,future_Away_game_percent_day_window2_a,future_home_game_percent_day_window2_a,future_Away_game_count_day_window5_a,future_home_game_count_day_window5_a,future_game_count_day_window5_a,future_Away_game_percent_day_window5_a,future_home_game_percent_day_window5_a,future_Away_game_count_day_window8_a,future_home_game_count_day_window8_a,future_game_count_day_window8_a,future_Away_game_percent_day_window8_a,future_home_game_percent_day_window8_a,future_Away_game_count_day_window12_a,future_home_game_count_day_window12_a,future_game_count_day_window12_a,future_Away_game_percent_day_window12_a,future_home_game_percent_day_window12_a,future_Away_game_count_day_window999_a,future_home_game_count_day_window999_a,future_game_count_day_window999_a,future_Away_game_percent_day_window999_a,future_home_game_percent_day_window999_a,team_id_b,away_home__b,latitude_b,longitude_b,game_latitude_b,game_longitude_b,travel_distance_b,game_time_zone_b,time_zone_b,time_zone_diff_b,time_zone_diff_abs_b,city_proper_metro_Area_b,Country/Region_b,city_proper_metro_Area_GDP_bil_b,city_proper_metro_Area_GDP_bil1_b,game_date_diff_b,game_date_diff1_b,games_back_to_back_count_game_window2_b,games_back_to_back_count_game_window5_b,games_back_to_back_count_game_window12_b,games_back_to_back_count_game_window999_b,games_back_to_back_percentage_game_window2_b,games_back_to_back_percentage_game_window5_b,games_back_to_back_percentage_game_window12_b,games_back_to_back_percentage_game_window999_b,away_game_count_day_window2_b,home_game_count_day_window2_b,game_count_day_window2_b,away_game_percent_day_window2_b,home_game_percent_day_window2_b,away_game_count_day_window5_b,home_game_count_day_window5_b,game_count_day_window5_b,away_game_percent_day_window5_b,home_game_percent_day_window5_b,away_game_count_day_window8_b,home_game_count_day_window8_b,game_count_day_window8_b,away_game_percent_day_window8_b,home_game_percent_day_window8_b,away_game_count_day_window12_b,home_game_count_day_window12_b,game_count_day_window12_b,away_game_percent_day_window12_b,home_game_percent_day_window12_b,away_game_count_day_window999_b,home_game_count_day_window999_b,game_count_day_window999_b,away_game_percent_day_window999_b,home_game_percent_day_window999_b,future_Away_game_count_day_window2_b,future_home_game_count_day_window2_b,future_game_count_day_window2_b,future_Away_game_percent_day_window2_b,future_home_game_percent_day_window2_b,future_Away_game_count_day_

In [25]:
def get_numeric_diff(df_, groupby_list_, not_difference_list_, suffix_list_):
    
    column_names_numeric_list_ = list(df_.select_dtypes('number').columns)
    
    column_names_numeric_list_cleaned_ = [k for k in column_names_numeric_list_
                                          if (not k in not_difference_list_)]
    
    column_names_first_second_list_ = [k.split(suffix_list_[0])[0]
                                       for k in column_names_numeric_list_cleaned_
                                       if k.endswith(suffix_list_[0])]
    
    column_names_first_second_list_rgl_ = [k for k in column_names_first_second_list_
                                           if not k in groupby_list_]

    column_names_first_second_list_rgl_first_ = [k + suffix_list_[0] \
                                                 for k in column_names_first_second_list_rgl_]
    
    column_names_first_second_list_rgl_second_ = [k + suffix_list_[1] \
                                                  for k in column_names_first_second_list_rgl_]
    
    column_names_first_second_list_rgl_diff_ = [k + '_diff' \
                                                for k in column_names_first_second_list_rgl_]
    
    #return len(column_names_first_second_list_rgl_first_), len(column_names_first_second_list_rgl_second_), len(column_names_first_second_list_rgl_diff_)
    
    
    df_[column_names_first_second_list_rgl_diff_] = df_[column_names_first_second_list_rgl_first_]\
                                                        .fillna(0).values - \
                                                        df_[column_names_first_second_list_rgl_second_]\
                                                        .fillna(0)
    
    column_names_drop_ = column_names_first_second_list_rgl_first_ + column_names_first_second_list_rgl_second_
    
    df_.drop(columns = column_names_drop_, inplace=True)    
    
    #return df_[column_names_first_second_list_rgl_first_]
    
    #return df_[column_names_first_second_list_rgl_second_]
    
    return df_

In [26]:
###########give me arguments please
df_differences = \
get_numeric_diff(df_71C_merge_71A_71B_2010_2018_wide.drop(columns=['time_zone_diff_b', 'time_zone_diff_abs_b']),
                 ['team_id', 'season'],
                 ['game_id', 'game_date', 'season', 
                  'season_type', 'season_year', 'time_zone_diff_a', 
                  'time_zone_diff_abs_a'],
                 ['_a', '_b'])

p(df_differences)

(10266, 83)


,game_id,team_id_a,game_date,season,game_time_zone_a,time_zone_a,time_zone_diff_a,time_zone_diff_abs_a,city_proper_metro_Area_a,Country/Region_a,team_id_b,game_time_zone_b,time_zone_b,city_proper_metro_Area_b,Country/Region_b,away_home__diff,latitude_diff,longitude_diff,game_latitude_diff,game_longitude_diff,travel_distance_diff,city_proper_metro_Area_GDP_bil_diff,city_proper_metro_Area_GDP_bil1_diff,game_date_diff_diff,game_date_diff1_diff,games_back_to_back_count_game_window2_diff,games_back_to_back_count_game_window5_diff,games_back_to_back_count_game_window12_diff,games_back_to_back_count_game_window999_diff,games_back_to_back_percentage_game_window2_diff,games_back_to_back_percentage_game_window5_diff,games_back_to_back_percentage_game_window12_diff,games_back_to_back_percentage_game_window999_diff,away_game_count_day_window2_diff,home_game_count_day_window2_diff,game_count_day_window2_diff,away_game_percent_day_window2_diff,home_game_percent_day_window2_diff,away_game_count_day_window5_diff,home_game_count_day_window5_diff,game_count_day_window5_diff,away_game_percent_day_window5_diff,home_game_percent_day_window5_diff,away_game_count_day_window8_diff,home_game_count_day_window8_diff,game_count_day_window8_diff,away_game_percent_day_window8_diff,home_game_percent_day_window8_diff,away_game_count_day_window12_diff,home_game_count_day_window12_diff,game_count_day_window12_diff,away_game_percent_day_window12_diff,home_game_percent_day_window12_diff,away_game_count_day_window999_diff,home_game_count_day_window999_diff,game_count_day_window999_diff,away_game_percent_day_window999_diff,home_game_percent_day_window999_diff,future_Away_game_count_day_window2_diff,future_home_game_count_day_window2_diff,future_game_count_day_window2_diff,future_Away_game_percent_day_window2_diff,future_home_game_percent_day_window2_diff,future_Away_game_count_day_window5_diff,future_home_game_count_day_window5_diff,future_game_count_day_window5_diff,future_Away_game_percent_day_window5_diff,future_home_game_percent_day_window5_diff,future_Away_game_count_day_window8_diff,future_home_game_count_day_window8_diff,future_game_count_day_window8_diff,future_Away_game_percent_day_window8_diff,future_home_game_percent_day_window8_diff,future_Away_game_count_day_window12_diff,future_home_game_count_day_window12_diff,future_game_count_day_window12_diff,future_Away_game_percent_day_window12_diff,future_home_game_percent_day_window12_diff,future_Away_game_count_day_window999_diff,future_home_game_count_day_window999_diff,future_game_count_day_window999_diff,future_Away_game_percent_day_window999_diff,future_home_game_percent_day_window999_diff
0,21000001,1610612738,2010-10-26,2010-11,America/New_York,America/New_York,0,0,Boston,United States,1610612748,America/New_York,America/New_York,Miami,United States,1.0,16.586081,9.135329,0.0,0.0,-1257.755091,86.070,297.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-1.0,1.0,0.0,-1.0,1.0,-1.0,1.0,0.0,-1.000000,1.000000,-1.0,1.0,0.0,-1.000000,1.000000,-1.0,1.0,0.0,-1.000000,1.000000,-1.0,1.0,0.0,-1.000000,1.000000,-1.0,1.0,0.0,-0.5,0.5,-1.0,1.0,0.0,-0.333333,0.333333,-1.0,0.0,-1.0,-0.100000,0.100000,-2.0,1.0,-1.0,-0.238095,0.238095,0.0,0.0,0.0,0.000000,0.000000
1,21000002,1610612756,2010-10-26,2010-11,America/Los_Angeles,America/Phoenix,0,0,Phoenix,United States,1610612757,America/Los_Angeles,America/Los_Angeles,Portland,United States,-1.0,-12.071810,10.600053,0.0,0.0,1005.606855,90.792,448.60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,-1.0,0.0,1.0,-1.0,1.0,-1.0,0.0,1.000000,-1.000000,1.0,-1.0,0.0,1.000000,-1.000000,1.0,-1.0,0.0,1.000000,-1.000000,1.0,-1.0,0.0,1.000000,-1.000000,0.0,-1.0,-1.0,0.5,-0.5,0.0,0.0,0.0,0.000000,0.000000,-2.0,0.0,-2.0,-0.133333,0.133333,-2.0,0.0,-2.0,-0.171429,0.171429,0.0,0.0,0.0,0.000000,0.000000
2,21000003,1610612745,2010-10-26,2010-11,America/Los_Angeles,America/Chicago,-2,2,Houston,United States,1610612747,America/Los_Angeles,America/Los_Angeles,Los Angeles,United States,-1.0,-4.29

In [27]:
datapath = os.path.join('..', 'processed_data')

if not os.path.exists(datapath):
    os.mkdir(datapath)

datapath_df = os.path.join(datapath, '72_71C_schedule_location_differences_2010_2018.csv')
if not os.path.exists(datapath_df):
    df_differences.to_csv(datapath_df, index=False)

In [28]:
df_72_71C_schedule_location_differences_2010_2018_readback = \
rcp('72_71C_schedule_location_differences_2010_2018.csv')

p(df_72_71C_schedule_location_differences_2010_2018_readback)

(10266, 83)


,game_id,team_id_a,game_date,season,game_time_zone_a,time_zone_a,time_zone_diff_a,time_zone_diff_abs_a,city_proper_metro_Area_a,Country/Region_a,team_id_b,game_time_zone_b,time_zone_b,city_proper_metro_Area_b,Country/Region_b,away_home__diff,latitude_diff,longitude_diff,game_latitude_diff,game_longitude_diff,travel_distance_diff,city_proper_metro_Area_GDP_bil_diff,city_proper_metro_Area_GDP_bil1_diff,game_date_diff_diff,game_date_diff1_diff,games_back_to_back_count_game_window2_diff,games_back_to_back_count_game_window5_diff,games_back_to_back_count_game_window12_diff,games_back_to_back_count_game_window999_diff,games_back_to_back_percentage_game_window2_diff,games_back_to_back_percentage_game_window5_diff,games_back_to_back_percentage_game_window12_diff,games_back_to_back_percentage_game_window999_diff,away_game_count_day_window2_diff,home_game_count_day_window2_diff,game_count_day_window2_diff,away_game_percent_day_window2_diff,home_game_percent_day_window2_diff,away_game_count_day_window5_diff,home_game_count_day_window5_diff,game_count_day_window5_diff,away_game_percent_day_window5_diff,home_game_percent_day_window5_diff,away_game_count_day_window8_diff,home_game_count_day_window8_diff,game_count_day_window8_diff,away_game_percent_day_window8_diff,home_game_percent_day_window8_diff,away_game_count_day_window12_diff,home_game_count_day_window12_diff,game_count_day_window12_diff,away_game_percent_day_window12_diff,home_game_percent_day_window12_diff,away_game_count_day_window999_diff,home_game_count_day_window999_diff,game_count_day_window999_diff,away_game_percent_day_window999_diff,home_game_percent_day_window999_diff,future_Away_game_count_day_window2_diff,future_home_game_count_day_window2_diff,future_game_count_day_window2_diff,future_Away_game_percent_day_window2_diff,future_home_game_percent_day_window2_diff,future_Away_game_count_day_window5_diff,future_home_game_count_day_window5_diff,future_game_count_day_window5_diff,future_Away_game_percent_day_window5_diff,future_home_game_percent_day_window5_diff,future_Away_game_count_day_window8_diff,future_home_game_count_day_window8_diff,future_game_count_day_window8_diff,future_Away_game_percent_day_window8_diff,future_home_game_percent_day_window8_diff,future_Away_game_count_day_window12_diff,future_home_game_count_day_window12_diff,future_game_count_day_window12_diff,future_Away_game_percent_day_window12_diff,future_home_game_percent_day_window12_diff,future_Away_game_count_day_window999_diff,future_home_game_count_day_window999_diff,future_game_count_day_window999_diff,future_Away_game_percent_day_window999_diff,future_home_game_percent_day_window999_diff
0,21000001,1610612738,2010-10-26,2010-11,America/New_York,America/New_York,0,0,Boston,United States,1610612748,America/New_York,America/New_York,Miami,United States,1.0,16.586081,9.135329,0.0,0.0,-1257.755091,86.070,297.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-1.0,1.0,0.0,-1.0,1.0,-1.0,1.0,0.0,-1.000000,1.000000,-1.0,1.0,0.0,-1.000000,1.000000,-1.0,1.0,0.0,-1.000000,1.000000,-1.0,1.0,0.0,-1.000000,1.000000,-1.0,1.0,0.0,-0.5,0.5,-1.0,1.0,0.0,-0.333333,0.333333,-1.0,0.0,-1.0,-0.100000,0.100000,-2.0,1.0,-1.0,-0.238095,0.238095,0.0,0.0,0.0,0.000000,0.000000
1,21000002,1610612756,2010-10-26,2010-11,America/Los_Angeles,America/Phoenix,0,0,Phoenix,United States,1610612757,America/Los_Angeles,America/Los_Angeles,Portland,United States,-1.0,-12.071810,10.600053,0.0,0.0,1005.606855,90.792,448.60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,-1.0,0.0,1.0,-1.0,1.0,-1.0,0.0,1.000000,-1.000000,1.0,-1.0,0.0,1.000000,-1.000000,1.0,-1.0,0.0,1.000000,-1.000000,1.0,-1.0,0.0,1.000000,-1.000000,0.0,-1.0,-1.0,0.5,-0.5,0.0,0.0,0.0,0.000000,0.000000,-2.0,0.0,-2.0,-0.133333,0.133333,-2.0,0.0,-2.0,-0.171429,0.171429,0.0,0.0,0.0,0.000000,0.000000
2,21000003,1610612745,2010-10-26,2010-11,America/Los_Angeles,America/Chicago,-2,2,Houston,United States,1610612747,America/Los_Angeles,America/Los_Angeles,Los Angeles,United States,-1.0,-4.29